<a href="https://colab.research.google.com/github/jana-nf/Estagio_CBL_CienciaDeDados_HDL/blob/main/Estagio_CBL_CienciaDeDados_HDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estagio_CBL_CiênciaDeDados_HDL
🎯 Protótipo de pipeline de dados (ETL/ELT) desenvolvido com Python, Pandas, SQL e GIT, seguindo a metodologia Challenge-Based Learning (CBL) para estágio em Ciência de Dados(HDL).

###Como os Dados Brutos são Transformados em Insights Acionáveis e Conhecimento Estratégico para a Empresa.

###Construir um mini-pipeline de ETL (Extração, Transformação e Carga) para processar dados de vendas (simulados: CSV, JSON, consulta SQL) e gerar um painel de indicadores (dashboard) simples em Python, que revele a principal 'dor' ou oportunidade de otimização de processo na operação de uma empresa.

#Dataset do Projeto
Dataset Público de E-commerce Brasileiro da Olist (Kaggle): https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?resource=download


#Construção do Mini-Pipeline de ETL
Extração (E): Criar três fontes de dados de acordo com o dataset do projeto: um CSV de pedidos, um JSON de informações do cliente (extraído via simulação de API), e um pequeno banco de dados SQLite com dados de estoque (consulta SQL básica).

In [185]:
import pandas as pd
import json
import sqlite3
import os

###Dataset OLIST_ORDERS_DATASET.CSV para JSON

In [142]:
BASE_DIR = os.path.join(os.getcwd())
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')
INPUT_CSV = 'olist_orders_dataset.csv'
OUTPUT_JSON = 'orders_api.json'

def convert_csv_to_json():
    """Lê o CSV de pedidos e salva como JSON para simular uma fonte de API."""

    path_input_csv = os.path.join(RAW_DATA_PATH, INPUT_CSV)
    path_output_json = os.path.join(RAW_DATA_PATH, OUTPUT_JSON)

    print(f"Lendo CSV de pedidos em: {path_input_csv}")

    try:
        # Lê o CSV original
        df_customers = pd.read_csv(path_input_csv)

        # O argumento 'orient' define a estrutura do JSON.
        # 'records' é um formato comum para APIs, onde cada linha é um objeto JSON.
        df_customers.to_json(
            path_output_json,
            orient='records',
            date_format='iso' # Formato padrão para datas, se houver
        )

        print(f"Sucesso! {len(df_customers)} registros convertidos.")
        print(f"Arquivo JSON salvo em: {path_output_json}")

    except Exception as e:
        print(f"Ocorreu um erro durante a conversão: {e}")

if __name__ == "__main__":
    convert_csv_to_json()

Lendo CSV de pedidos em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/01_raw/olist_orders_dataset.csv
Sucesso! 99441 registros convertidos.
Arquivo JSON salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/01_raw/orders_api.json


###Dataset OLIST_ORDER_ITEMS_DATASET.CSV para SQLite_db

In [143]:
BASE_DIR = os.path.join(os.getcwd())
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')

# 1. Nomes dos arquivos e tabelas
INPUT_CSV_ITEMS = 'olist_order_items_dataset.csv'
OUTPUT_DB_NAME = 'estoque.db'
SQL_TABLE_NAME = 'order_items' # Nome da tabela dentro do banco de dados

def create_sqlite_db():
    """Lê o CSV de itens e carrega os dados em uma tabela SQLite local."""

    path_input_csv = os.path.join(RAW_DATA_PATH, INPUT_CSV_ITEMS)
    path_output_db = os.path.join(RAW_DATA_PATH, OUTPUT_DB_NAME)

    # Verifica se o arquivo CSV de origem existe
    if not os.path.exists(path_input_csv):
        print(f"ERRO: Arquivo de itens não encontrado em: {path_input_csv}")
        print("Certifique-se de que 'olist_order_items_dataset.csv' está na pasta 01_raw.")
        return

    print(f"Lendo CSV de itens em: {path_input_csv}")

    try:
        # Lê o CSV original com Pandas
        df_items = pd.read_csv(path_input_csv)

        # Conecta-se ao banco de dados SQLite (ele será criado se não existir)
        conn = sqlite3.connect(path_output_db)

        print(f"Carregando {len(df_items)} registros na tabela '{SQL_TABLE_NAME}'...")

        # Carrega o DataFrame inteiro para a tabela SQLite
        # if_exists='replace' garante que ele substitua se você rodar o script novamente
        df_items.to_sql(
            name=SQL_TABLE_NAME,
            con=conn,
            if_exists='replace',
            index=False
        )

        # Confirmação (Opcional: testa se a tabela foi criada)
        cursor = conn.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {SQL_TABLE_NAME}")
        count = cursor.fetchone()[0]

        conn.close()

        print(f"Sucesso! Banco de dados '{OUTPUT_DB_NAME}' criado com {count} linhas na tabela.")
        print(f"Arquivo DB salvo em: {path_output_db}")

    except Exception as e:
        print(f"Ocorreu um erro ao criar o banco de dados SQLite: {e}")

if __name__ == "__main__":
    create_sqlite_db()

Lendo CSV de itens em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/01_raw/olist_order_items_dataset.csv
Carregando 112650 registros na tabela 'order_items'...
Sucesso! Banco de dados 'estoque.db' criado com 112650 linhas na tabela.
Arquivo DB salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/01_raw/estoque.db


In [144]:
import pandas as pd
import os

# Assume que você já está no diretório raiz do projeto e que as pastas estão configuradas.
RAW_DATA_PATH = os.path.join(os.getcwd(), 'data', '01_raw')

INPUT_CSV = 'olist_customers_dataset.csv'
OUTPUT_JSON = 'clientes_api.json'

path_input_csv = os.path.join(RAW_DATA_PATH, INPUT_CSV)
path_output_json = os.path.join(RAW_DATA_PATH, OUTPUT_JSON)

if os.path.exists(path_input_csv):
    print(f"Lendo CSV de clientes: {INPUT_CSV}")

    # 1. Lê o CSV
    df_customers = pd.read_csv(path_input_csv)

    # 2. Salva como JSON (orient='records' simula o formato comum de API)
    df_customers.to_json(
        path_output_json,
        orient='records'
    )

    print(f"✅ Arquivo JSON criado com sucesso: {OUTPUT_JSON}")
else:
    print(f"⚠️ ERRO: {INPUT_CSV} não encontrado em {RAW_DATA_PATH}")

Lendo CSV de clientes: olist_customers_dataset.csv
✅ Arquivo JSON criado com sucesso: clientes_api.json


In [145]:
import pandas as pd
import sqlite3
import os

# Assume que você já está no diretório raiz do projeto.
RAW_DATA_PATH = os.path.join(os.getcwd(), 'data', '01_raw')

INPUT_CSV_ITEMS = 'olist_order_items_dataset.csv'
OUTPUT_DB_NAME = 'estoque.db'
SQL_TABLE_NAME = 'order_items'

path_input_csv = os.path.join(RAW_DATA_PATH, INPUT_CSV_ITEMS)
path_output_db = os.path.join(RAW_DATA_PATH, OUTPUT_DB_NAME)

if os.path.exists(path_input_csv):
    print(f"\nLendo CSV de itens: {INPUT_CSV_ITEMS}")

    # 1. Lê o CSV
    df_items = pd.read_csv(path_input_csv)

    # 2. Conecta ao DB (cria se não existir)
    conn = sqlite3.connect(path_output_db)

    # 3. Carrega o DataFrame para o DB
    df_items.to_sql(
        name=SQL_TABLE_NAME,
        con=conn,
        if_exists='replace',
        index=False
    )

    conn.close()
    print(f"✅ Banco de dados SQLite criado com sucesso: {OUTPUT_DB_NAME}")
else:
    print(f"⚠️ ERRO: {INPUT_CSV_ITEMS} não encontrado em {RAW_DATA_PATH}")


Lendo CSV de itens: olist_order_items_dataset.csv
✅ Banco de dados SQLite criado com sucesso: estoque.db


###Salvar no Google Drive

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
# Mude este caminho para o local exato da sua pasta raiz no Drive
PROJECT_ROOT = '/content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL'

# Navega (Change Directory) para a pasta raiz do projeto
import os
os.chdir(PROJECT_ROOT)

print(f"Diretório de trabalho atual: {os.getcwd()}")

Diretório de trabalho atual: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL


In [148]:
# Instala as bibliotecas
!pip install pandas matplotlib

# Cria a estrutura de pastas, caso não exista
!mkdir -p data/01_raw data/03_clean src reports/figures

print("Setup do ambiente concluído!")

Setup do ambiente concluído!


###Execução do Pipeline ETL (Extração, Transformação e Carga)

In [149]:
%%writefile src/etl_pipeline.py

import pandas as pd
import sqlite3
import os

# --- Configurações de Caminho (Robusto para Notebook/Script) ---
try:
    # Tenta definir o caminho base usando '__file__' (para quando rodar via terminal)
    BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..')
    print("Modo de execução: Script (.py)")
except NameError:
    # Se falhar (Colab/Notebook), assume que o diretório atual é a pasta raiz
    BASE_DIR = os.getcwd()
    print("Modo de execução: Interativo (Notebook/Console)")

RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')
CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')

# --- Nomes dos Arquivos de Entrada ---
CSV_ORDERS = 'olist_orders_dataset.csv'
JSON_CUSTOMERS = 'clientes_api.json'
SQLITE_DB = 'estoque.db'
SQLITE_TABLE = 'order_items'
OUTPUT_FILE = 'clean_data.csv'

# --- 1. EXTRAÇÃO (E) ---

def extract_orders_csv():
    """Extrai dados de pedidos e logística do arquivo CSV."""
    path = os.path.join(RAW_DATA_PATH, CSV_ORDERS)
    if not os.path.exists(path):
        raise FileNotFoundError(f"CSV de Pedidos não encontrado: {path}")
    return pd.read_csv(path)

def extract_customers_json():
    """Extrai dados de clientes (simulação de API) do arquivo JSON."""
    path = os.path.join(RAW_DATA_PATH, JSON_CUSTOMERS)
    if not os.path.exists(path):
        raise FileNotFoundError(f"JSON de Clientes não encontrado: {path}")
    return pd.read_json(path, orient='records')

def extract_items_sql():
    """Extrai dados de itens/estoque via consulta SQL no SQLite."""
    path = os.path.join(RAW_DATA_PATH, SQLITE_DB)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Banco de dados SQLite não encontrado: {path}")

    conn = sqlite3.connect(path)
    query = f"SELECT order_id, product_id, price, freight_value FROM {SQLITE_TABLE}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# --- 2. TRANSFORMAÇÃO (T) ---

def transform_data(df_orders, df_customers, df_items):
    """Realiza limpeza, padronização de tipos e a unificação dos dados."""
    print("Iniciando Transformação de dados...")

    # Padronização de Tipos (Datas)
    date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date']
    for col in date_cols:
        df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')

    # Unificação (Merge)
    df_merged = pd.merge(df_orders, df_customers[['customer_id', 'customer_state']], on='customer_id', how='left')
    df_final = pd.merge(df_merged, df_items, on='order_id', how='left')
    print(f"Dataset unificado criado com {len(df_final)} linhas.")

    # Cálculo da Métrica Central
    df_final['delivery_time_days'] = (
        df_final['order_delivered_customer_date'] - df_final['order_purchase_timestamp']
    ).dt.days

    # Tratamento de Valores Ausentes (NaN)
    df_clean = df_final.dropna(subset=['order_delivered_customer_date', 'product_id'])
    print(f"Registros válidos após limpeza crítica: {len(df_clean)} linhas.")

    return df_clean

# --- 3. CARGA (L) ---

def load_data(df_clean):
    """Carrega o dataset limpo para o diretório de análise."""
    os.makedirs(CLEAN_DATA_PATH, exist_ok=True)

    final_path = os.path.join(CLEAN_DATA_PATH, OUTPUT_FILE)
    df_clean.to_csv(final_path, index=False)

    print(f"\n--- CARGA CONCLUÍDA ---")
    print(f"Dataset limpo e pronto para análise salvo em: {final_path}")

# --- FUNÇÃO PRINCIPAL ---

def run_etl_pipeline():
    """Função principal que orquestra todo o pipeline ETL."""
    try:
        print("--- INÍCIO DO PIPELINE ETL ---")

        # E: Extração
        df_orders = extract_orders_csv()
        df_customers = extract_customers_json()
        df_items = extract_items_sql()

        # T: Transformação
        df_clean = transform_data(df_orders, df_customers, df_items)

        # L: Carga
        load_data(df_clean)

        print("--- PIPELINE ETL EXECUTADO COM SUCESSO! ---")

    except FileNotFoundError as e:
        print(f"\nERRO FATAL: {e}")
        print("Verifique se todos os arquivos CSV, JSON e DB estão na pasta 'data/01_raw/'.")
    except Exception as e:
        print(f"\nERRO INESPERADO NO PIPELINE: {e}")


if __name__ == "__main__":
    run_etl_pipeline()

Overwriting src/etl_pipeline.py


In [150]:
%%writefile src/analysis_report.py

import pandas as pd
import matplotlib.pyplot as plt
import os

# --- Configurações de Caminho (Robusto para Colab) ---
try:
    BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..')
except NameError:
    BASE_DIR = os.getcwd()

CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')
REPORTS_PATH = os.path.join(BASE_DIR, 'reports', 'figures')

def run_analysis():
    """Realiza a análise estratégica, gera o insight e o relatório."""
    print("--- 4. INÍCIO DA ANÁLISE E EXTRAÇÃO DE INSIGHTS ---")

    # 1. Carga dos Dados
    path_clean_data = os.path.join(CLEAN_DATA_PATH, 'clean_data.csv')
    df_clean = pd.read_csv(path_clean_data, parse_dates=['order_purchase_timestamp', 'order_delivered_customer_date'])

    path_products = os.path.join(RAW_DATA_PATH, 'olist_products_dataset.csv')
    df_products = pd.read_csv(path_products)

    # 2. Junção com Categoria de Produto
    df_analysis = pd.merge(
        df_clean,
        df_products[['product_id', 'product_category_name']],
        on='product_id',
        how='left'
    )

    # 3. Análise de Desempenho
    df_summary = df_analysis.groupby('product_category_name').agg(
        avg_delivery_time=('delivery_time_days', 'mean'),
        total_price=('price', 'sum')
    ).reset_index()

    # Encontra a "DOR" Logística (Top 10 mais vendidas com maior tempo de entrega)
    top_10_categories = df_summary.sort_values(by='total_price', ascending=False).head(10)
    pain_category = top_10_categories.sort_values(by='avg_delivery_time', ascending=False).iloc[0]

    # 4. Geração do Insight Estratégico
    INSIGHT = (
        f"A principal 'dor' logística é encontrada na categoria de produto '{pain_category['product_category_name']}'. "
        f"Tempo médio de entrega: {pain_category['avg_delivery_time']:.2f} dias. Otimizar esta categoria é crucial."
    )
    print("\n--- INSIGHT ENCONTRADO ---")
    print(INSIGHT)

    # 5. Geração do Relatório Visual
    plt.figure(figsize=(12, 6))
    plt.barh(top_10_categories['product_category_name'], top_10_categories['avg_delivery_time'],
             color=['red' if cat == pain_category['product_category_name'] else 'skyblue' for cat in top_10_categories['product_category_name']])

    plt.xlabel('Tempo Médio de Entrega (Dias)')
    plt.title('Tempo Médio de Entrega das TOP 10 Categorias por Volume')
    plt.tight_layout()

    os.makedirs(REPORTS_PATH, exist_ok=True)
    report_path = os.path.join(REPORTS_PATH, 'delivery_pain_point_report.png')
    plt.savefig(report_path)
    plt.close()

    print(f"\nRelatório visual salvo em: {report_path}")

if __name__ == "__main__":
    run_analysis()

Overwriting src/analysis_report.py


In [151]:
# Isso criará o arquivo data/03_clean/clean_data.csv
!python src/etl_pipeline.py

Modo de execução: Script (.py)
--- INÍCIO DO PIPELINE ETL ---
Iniciando Transformação de dados...
Dataset unificado criado com 113425 linhas.
Registros válidos após limpeza crítica: 110196 linhas.

--- CARGA CONCLUÍDA ---
Dataset limpo e pronto para análise salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/src/../data/03_clean/clean_data.csv
--- PIPELINE ETL EXECUTADO COM SUCESSO! ---


###Rodar o ETL

In [152]:
!python src/etl_pipeline.py

Modo de execução: Script (.py)
--- INÍCIO DO PIPELINE ETL ---
Iniciando Transformação de dados...
Dataset unificado criado com 113425 linhas.
Registros válidos após limpeza crítica: 110196 linhas.

--- CARGA CONCLUÍDA ---
Dataset limpo e pronto para análise salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/src/../data/03_clean/clean_data.csv
--- PIPELINE ETL EXECUTADO COM SUCESSO! ---


In [153]:
!ls src/

analysis_report.py  etl_pipeline.py


In [154]:
# Use o comando de execução novamente
# Use o caminho absoluto para máxima confiabilidade no Colab
!python {os.getcwd()}/src/etl_pipeline.py

Modo de execução: Script (.py)
--- INÍCIO DO PIPELINE ETL ---
Iniciando Transformação de dados...
Dataset unificado criado com 113425 linhas.
Registros válidos após limpeza crítica: 110196 linhas.

--- CARGA CONCLUÍDA ---
Dataset limpo e pronto para análise salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/src/../data/03_clean/clean_data.csv
--- PIPELINE ETL EXECUTADO COM SUCESSO! ---


#Transformação (T):
Escrever um script Python (usando Pandas) que leia, integre (merge), limpe (trate NaNs e formate tipos) e transforme os dados das três fontes em um dataset único e coerente.



In [155]:
import pandas as pd
# Assume que df_orders, df_customers, e df_items foram extraídos com sucesso.

def transform_data(df_orders, df_customers, df_items):
    """
    Realiza a limpeza, padronização de tipos e a unificação dos dados
    das três fontes em um dataset único e coerente.
    """
    print("Iniciando Transformação de dados...")

    # --- 1. PADRONIZAÇÃO DE TIPOS (Data/Tempo) ---
    # Passo T1: Conversão das colunas de data para o tipo datetime.
    date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
    for col in date_cols:
        # errors='coerce' transforma valores inválidos em NaT (que serão tratados mais tarde).
        df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')

    # --- 2. UNIFICAÇÃO (MERGE) ---
    # Passo T2: Integração dos DataFrames usando chaves primárias/estrangeiras.

    # Merge 1: Pedidos + Clientes (chave: 'customer_id')
    # Usamos LEFT JOIN para manter todos os registros de pedidos mesmo que a informação do cliente falhe.
    df_merged = pd.merge(
        df_orders,
        df_customers[['customer_id', 'customer_state', 'customer_city']],
        on='customer_id',
        how='left'
    )

    # Merge 2: Resultado anterior + Itens (chave: 'order_id')
    # Este merge expande as linhas se houver vários itens por pedido.
    df_final = pd.merge(
        df_merged,
        df_items[['order_id', 'product_id', 'price', 'freight_value']],
        on='order_id',
        how='left'
    )
    print(f"Dataset unificado criado: {len(df_final)} linhas.")

    # --- 3. TRANSFORMAÇÃO DE VARIÁVEIS (Cálculo da Métrica Central) ---
    # Passo T3: Criar a métrica de interesse para o insight de logística.

    # Calcular o Tempo de Entrega Real (em dias)
    df_final['delivery_time_days'] = (
        df_final['order_delivered_customer_date'] - df_final['order_purchase_timestamp']
    ).dt.total_seconds() / (60*60*24)

    # Calcular o Desvio de Prazo (entrega real vs. estimada)
    df_final['delivery_deviation_days'] = (
        df_final['order_delivered_customer_date'] - df_final['order_estimated_delivery_date']
    ).dt.total_seconds() / (60*60*24)

    # --- 4. LIMPEZA FINAL (Tratamento de NaNs e Outliers) ---
    # Passo T4: Remover dados que não servem para a análise final.

    # Tratar NaNs: Remove registros onde as métricas centrais são nulas
    # (Ex: pedidos cancelados ou sem informação de produto/preço).
    df_clean = df_final.dropna(subset=['delivery_time_days', 'product_id', 'price'])

    # Tratar Outliers: Remove tempos de entrega não realistas (negativos ou muito longos)
    df_clean = df_clean[
        (df_clean['delivery_time_days'] > 0) &
        (df_clean['delivery_time_days'] < 200) # Filtro heurístico
    ]

    print(f"Registros válidos após limpeza e filtro de outliers: {len(df_clean)} linhas.")

    return df_clean

In [156]:
import pandas as pd
import os

# --- Configurações de Caminho ---
# Assumindo que você já navegou para a pasta raiz do projeto usando os.chdir()
BASE_DIR = os.getcwd()
CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')
OUTPUT_FILE = 'clean_data.csv'

# Caminho completo para o arquivo limpo
path_clean_data = os.path.join(CLEAN_DATA_PATH, OUTPUT_FILE)

print(f"Carregando o dataset final de: {path_clean_data}")

# 1. Carrega o DataFrame final
try:
    df_final = pd.read_csv(path_clean_data)

    print(f"Total de registros no dataset final: {len(df_final)}")
    print("\n--- Amostra da Tabela Final (5 primeiras linhas) ---")

    # 2. Exibe uma amostra usando a função 'head()'
    # Você verá uma tabela interativa no Colab.
    display(df_final.head())

    # Para mostrar as colunas mais importantes após a transformação:
    print("\n--- Colunas de Interesse (ID, Preço, Métrica) ---")
    display(df_final[[
        'order_id',
        'customer_state',
        'price',
        'delivery_time_days'
    ]].sample(5)) # Exibe 5 linhas aleatórias

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{OUTPUT_FILE}' não foi encontrado.")
    print("Execute o script 'src/etl_pipeline.py' primeiro para gerar a carga de dados.")

Carregando o dataset final de: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/03_clean/clean_data.csv
Total de registros no dataset final: 110196

--- Amostra da Tabela Final (5 primeiras linhas) ---


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_state,product_id,price,freight_value,delivery_time_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,SP,87285b34884572647811a353c7ac498a,29.99,8.72,8.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,BA,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,13.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,GO,aa4383b373c6aca5d8797843e5594415,159.90,19.22,9.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,RN,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20,13.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,SP,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72,2.0



--- Colunas de Interesse (ID, Preço, Métrica) ---


,order_id,customer_state,price,delivery_time_days
5331,b825b53d8136d85eb4cbe60a1bd275f5,SP,229.90,7.0
30791,398f1723ca5725d585fbc000ec3edb05,SP,137.90,3.0
13613,4cb7d866ce5acba93c58383df3081469,CE,109.90,16.0
5553,47dc3a1de5a0aec4323cdade0588655c,PR,399.00,9.0
83685,e3cd6d82ade0e89d757b4d900460f1ad,MG,46.86,11.0


###Tabela de Integração

In [157]:
import pandas as pd
import os
from IPython.display import display

# --- Configurações de Caminho ---
# Assumindo que você já navegou para a pasta raiz do projeto com os.chdir()
BASE_DIR = os.getcwd()
CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')
OUTPUT_FILE = 'clean_data.csv'

# Caminho completo para o arquivo limpo
path_clean_data = os.path.join(CLEAN_DATA_PATH, OUTPUT_FILE)

print(f"Carregando o dataset final de: {path_clean_data}")

# 1. Carrega o DataFrame final
try:
    df_final = pd.read_csv(path_clean_data)

    print(f"Total de registros no dataset final: {len(df_final)}")

    # Define as colunas que representam o resultado da integração e transformação:
    # order_id, product_id, customer_state (vindos de diferentes fontes)
    # price, delivery_time_days (transformação/cálculo)
    colunas_interesse = [
        'order_id',
        'product_id',
        'customer_state',
        'price',
        'freight_value',
        'delivery_time_days'
    ]

    print("\n--- Amostra da Tabela Final Integrada (5 linhas aleatórias) ---")

    # 2. Exibe as 5 primeiras linhas do DataFrame, focando nas colunas mais relevantes
    # O uso de display() renderiza a tabela interativa no Colab.
    display(df_final[colunas_interesse].sample(5))

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{OUTPUT_FILE}' não foi encontrado em {path_clean_data}")
    print("Verifique se o script 'src/etl_pipeline.py' foi executado com sucesso e gerou o arquivo na pasta 'data/03_clean/'.")

Carregando o dataset final de: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/03_clean/clean_data.csv
Total de registros no dataset final: 110196

--- Amostra da Tabela Final Integrada (5 linhas aleatórias) ---


,order_id,product_id,customer_state,price,freight_value,delivery_time_days
107993,df16628521a91b8dc473aa698aac7285,1c45430f3f3063bdb54d28e87824ed34,RJ,375.00,31.67,10.0
78421,7ff15ea63e622694922805d48de711a1,a4aa7c1427c31344e5f7cc3d839fe562,SP,110.00,20.26,8.0
8978,b80fc28f62c785fe351b7e4d70185334,06484143f7046fc1c2ec000aba13e28a,RJ,24.99,16.60,11.0
64482,be28f879f9c5e345df9e5b5fb441e6de,bd94ffbca2eff2c87f0ef59c000db3f6,SP,119.90,8.09,18.0
68058,50d9186d77acd4105e6dc1098ed0a28c,c2bcdb759a32342591497db4153af052,MG,369.00,16.75,8.0


#Automação:
Garantir que este processo seja executado por um único script modular.



In [158]:
# src/etl_pipeline.py

import pandas as pd
import sqlite3
import os

# --- Configurações de Caminho (Robusto para Colab/Scripts) ---
try:
    # Tenta definir o caminho base para rodar como script (.py)
    BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..')
except NameError:
    # Se falhar (rodando no Colab/Notebook), assume o diretório atual
    BASE_DIR = os.getcwd()

RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')
CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')

# --- Nomes dos Arquivos ---
CSV_ORDERS = 'olist_orders_dataset.csv'
JSON_CUSTOMERS = 'clientes_api.json'
SQLITE_DB = 'estoque.db'
SQLITE_TABLE = 'order_items'
OUTPUT_FILE = 'clean_data.csv'

# --- E: Extração ---
def extract_data():
    """Extrai dados das três fontes: CSV, JSON e SQLite."""
    print("--- 1. EXTRAÇÃO: Lendo fontes de dados... ---")

    # 1. Leitura do CSV de Pedidos
    df_orders = pd.read_csv(os.path.join(RAW_DATA_PATH, CSV_ORDERS))

    # 2. Leitura do JSON de Clientes
    df_customers = pd.read_json(os.path.join(RAW_DATA_PATH, JSON_CUSTOMERS), orient='records')

    # 3. Leitura do SQLite de Itens
    conn = sqlite3.connect(os.path.join(RAW_DATA_PATH, SQLITE_DB))
    query = f"SELECT order_id, product_id, price, freight_value FROM {SQLITE_TABLE}"
    df_items = pd.read_sql_query(query, conn)
    conn.close()

    return df_orders, df_customers, df_items

# --- T: Transformação ---
def transform_data(df_orders, df_customers, df_items):
    """Integra, limpa e calcula a métrica chave (delivery_time_days)."""
    print("--- 2. TRANSFORMAÇÃO: Unificando e limpando dados... ---")

    # 1. Padronização de Tipos (Datas)
    date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date']
    for col in date_cols:
        df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')

    # 2. Merge de Pedidos, Clientes e Itens
    df_merged = pd.merge(df_orders, df_customers[['customer_id', 'customer_state']], on='customer_id', how='left')
    df_final = pd.merge(df_merged, df_items, on='order_id', how='left')

    # 3. Cálculo da Métrica: Tempo de Entrega (dias)
    df_final['delivery_time_days'] = (
        df_final['order_delivered_customer_date'] - df_final['order_purchase_timestamp']
    ).dt.total_seconds() / (60*60*24)

    # 4. Limpeza Crítica: Remoção de NaNs (pedidos sem entrega, preço ou produto)
    df_clean = df_final.dropna(subset=['delivery_time_days', 'product_id', 'price'])
    df_clean = df_clean[(df_clean['delivery_time_days'] > 0) & (df_clean['delivery_time_days'] < 200)]

    print(f"Dataset final limpo pronto para análise com {len(df_clean)} linhas.")
    return df_clean

# --- L: Carga (do ETL) ---
def load_clean_data(df_clean):
    """Carrega o dataset limpo para o arquivo final (clean_data.csv)."""
    os.makedirs(CLEAN_DATA_PATH, exist_ok=True)
    final_path = os.path.join(CLEAN_DATA_PATH, OUTPUT_FILE)
    df_clean.to_csv(final_path, index=False)
    print(f"--- 3. CARGA: Dataset salvo em {final_path} ---")
    return final_path

def run_etl_pipeline():
    try:
        df_orders, df_customers, df_items = extract_data()
        df_clean = transform_data(df_orders, df_customers, df_items)
        load_clean_data(df_clean)
    except FileNotFoundError as e:
        print(f"\nERRO FATAL: {e}")
        print("Certifique-se de que os arquivos de dados brutos estão em 'data/01_raw/'.")

if __name__ == "__main__":
    run_etl_pipeline()

--- 1. EXTRAÇÃO: Lendo fontes de dados... ---
--- 2. TRANSFORMAÇÃO: Unificando e limpando dados... ---
Dataset final limpo pronto para análise com 110194 linhas.
--- 3. CARGA: Dataset salvo em /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/data/03_clean/clean_data.csv ---


#Carga e Análise (L & Insights):
Carregar o dataset limpo em um arquivo final (ex: clean_data.csv).



In [159]:
# src/analysis_report.py

import pandas as pd
import matplotlib.pyplot as plt
import os

# --- Configurações de Caminho ---
try:
    BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..')
except NameError:
    BASE_DIR = os.getcwd()

CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')
REPORTS_PATH = os.path.join(BASE_DIR, 'reports', 'figures')

def run_analysis_and_report():
    print("--- 4. INÍCIO DA ANÁLISE E GERAÇÃO DE INSIGHT ---")

    # 1. Carga do Dataset Limpo (Carga para Análise)
    path_clean_data = os.path.join(CLEAN_DATA_PATH, 'clean_data.csv')
    df_clean = pd.read_csv(path_clean_data)

    # 2. Carga do Dataset de Produtos (Para obter a Categoria)
    path_products = os.path.join(RAW_DATA_PATH, 'olist_products_dataset.csv')
    df_products = pd.read_csv(path_products)

    # 3. Integração com Categoria
    df_analysis = pd.merge(
        df_clean,
        df_products[['product_id', 'product_category_name']],
        on='product_id',
        how='left'
    )

    # 4. Cálculo da Métrica de Insight (Margem e Desempenho)
    # Margem Bruta Simples: price / (price + freight_value)
    df_analysis['gross_margin_ratio'] = df_analysis['price'] / (df_analysis['price'] + df_analysis['freight_value'])

    # Agrupamento para Análise
    df_summary = df_analysis.groupby('product_category_name').agg(
        avg_delivery_time=('delivery_time_days', 'mean'),
        avg_margin_ratio=('gross_margin_ratio', 'mean'),
        total_price_sum=('price', 'sum')
    ).reset_index()

    # 5. Filtragem e Classificação da "DOR"
    # Filtrar apenas categorias com volume relevante (ex: Top 20% em preço)
    volume_threshold = df_summary['total_price_sum'].quantile(0.80)
    df_relevant = df_summary[df_summary['total_price_sum'] >= volume_threshold]

    # Encontrar a categoria de maior dor: ALTA MARGEM (Max Ratio) e MAIOR TEMPO (Max Time)
    # Classificamos por margem decrescente e por tempo decrescente.
    # A primeira categoria no topo será a mais relevante para o insight.
    df_pain = df_relevant.sort_values(by=['avg_margin_ratio', 'avg_delivery_time'], ascending=[False, False])

    pain_category = df_pain.iloc[0]

    # 6. Geração do Insight Estratégico
    INSIGHT = (
        f"A **principal oportunidade de otimização (Dor Logística)** está na categoria '{pain_category['product_category_name']}'. "
        f"Esta categoria tem uma alta margem de {pain_category['avg_margin_ratio']:.2f}, "
        f"mas apresenta um tempo médio de entrega de **{pain_category['avg_delivery_time']:.2f} dias**, "
        f"indicando que a lentidão logística está corroendo a lucratividade potencial."
    )
    print("\n--- INSIGHT ESTRATÉGICO ENCONTRADO ---")
    print(INSIGHT)

    # 7. Geração do Relatório Visual
    plt.figure(figsize=(12, 6))
    plt.scatter(df_relevant['avg_delivery_time'], df_relevant['avg_margin_ratio'],
                alpha=0.7, color='skyblue')

    # Destaca o ponto de dor
    plt.scatter(pain_category['avg_delivery_time'], pain_category['avg_margin_ratio'],
                color='red', s=200, label=f"Dor Crítica: {pain_category['product_category_name']}")

    plt.title('Relação entre Margem Bruta e Tempo de Entrega (Top Volume)')
    plt.xlabel('Tempo Médio de Entrega (Dias)')
    plt.ylabel('Média da Razão de Margem Bruta (Price / Total)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    report_path = os.path.join(REPORTS_PATH, 'margin_vs_delivery_report.png')
    plt.savefig(report_path)
    plt.close()

    print(f"\nRelatório visual salvo em: {report_path}")

if __name__ == "__main__":
    run_analysis_and_report()

--- 4. INÍCIO DA ANÁLISE E GERAÇÃO DE INSIGHT ---

--- INSIGHT ESTRATÉGICO ENCONTRADO ---
A **principal oportunidade de otimização (Dor Logística)** está na categoria 'relogios_presentes'. Esta categoria tem uma alta margem de 0.87, mas apresenta um tempo médio de entrega de **12.66 dias**, indicando que a lentidão logística está corroendo a lucratividade potencial.

Relatório visual salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/reports/figures/margin_vs_delivery_report.png


Desenvolver o segundo script de análise para extrair um insight claro (ex: "Qual a categoria de produto com maior margem, mas com o maior tempo médio de entrega?") e gerar um gráfico de visualização (relatório simples).



In [160]:
# src/analysis_report.py

import pandas as pd
import matplotlib.pyplot as plt
import os

# --- Configurações de Caminho ---
try:
    BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..')
except NameError:
    BASE_DIR = os.getcwd()

CLEAN_DATA_PATH = os.path.join(BASE_DIR, 'data', '03_clean')
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', '01_raw')
REPORTS_PATH = os.path.join(BASE_DIR, 'reports', 'figures')

def run_analysis_and_report():
    print("--- 4. INÍCIO DA ANÁLISE E GERAÇÃO DE INSIGHT ---")

    # 1. Carga do Dataset Limpo (Carga para Análise)
    path_clean_data = os.path.join(CLEAN_DATA_PATH, 'clean_data.csv')
    df_clean = pd.read_csv(path_clean_data)

    # 2. Carga do Dataset de Produtos (Para obter a Categoria)
    path_products = os.path.join(RAW_DATA_PATH, 'olist_products_dataset.csv')
    df_products = pd.read_csv(path_products)

    # 3. Integração com Categoria
    df_analysis = pd.merge(
        df_clean,
        df_products[['product_id', 'product_category_name']],
        on='product_id',
        how='left'
    )

    # 4. Cálculo da Métrica de Insight (Margem e Desempenho)
    # Margem Bruta Simples: price / (price + freight_value)
    df_analysis['gross_margin_ratio'] = df_analysis['price'] / (df_analysis['price'] + df_analysis['freight_value'])

    # Agrupamento para Análise
    df_summary = df_analysis.groupby('product_category_name').agg(
        avg_delivery_time=('delivery_time_days', 'mean'),
        avg_margin_ratio=('gross_margin_ratio', 'mean'),
        total_price_sum=('price', 'sum')
    ).reset_index()

    # 5. Filtragem e Classificação da "DOR"
    # Filtrar apenas categorias com volume relevante (ex: Top 20% em preço)
    volume_threshold = df_summary['total_price_sum'].quantile(0.80)
    df_relevant = df_summary[df_summary['total_price_sum'] >= volume_threshold]

    # Encontrar a categoria de maior dor: ALTA MARGEM (Max Ratio) e MAIOR TEMPO (Max Time)
    # Classificamos por margem decrescente e por tempo decrescente.
    # A primeira categoria no topo será a mais relevante para o insight.
    df_pain = df_relevant.sort_values(by=['avg_margin_ratio', 'avg_delivery_time'], ascending=[False, False])

    pain_category = df_pain.iloc[0]

    # 6. Geração do Insight Estratégico
    INSIGHT = (
        f"A **principal oportunidade de otimização (Dor Logística)** está na categoria '{pain_category['product_category_name']}'. "
        f"Esta categoria tem uma alta margem de {pain_category['avg_margin_ratio']:.2f}, "
        f"mas apresenta um tempo médio de entrega de **{pain_category['avg_delivery_time']:.2f} dias**, "
        f"indicando que a lentidão logística está corroendo a lucratividade potencial."
    )
    print("\n--- INSIGHT ESTRATÉGICO ENCONTRADO ---")
    print(INSIGHT)

    # 7. Geração do Relatório Visual
    plt.figure(figsize=(12, 6))
    plt.scatter(df_relevant['avg_delivery_time'], df_relevant['avg_margin_ratio'],
                alpha=0.7, color='skyblue')

    # Destaca o ponto de dor
    plt.scatter(pain_category['avg_delivery_time'], pain_category['avg_margin_ratio'],
                color='red', s=200, label=f"Dor Crítica: {pain_category['product_category_name']}")

    plt.title('Relação entre Margem Bruta e Tempo de Entrega (Top Volume)')
    plt.xlabel('Tempo Médio de Entrega (Dias)')
    plt.ylabel('Média da Razão de Margem Bruta (Price / Total)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    report_path = os.path.join(REPORTS_PATH, 'margin_vs_delivery_report.png')
    plt.savefig(report_path)
    plt.close()

    print(f"\nRelatório visual salvo em: {report_path}")

if __name__ == "__main__":
    run_analysis_and_report()

--- 4. INÍCIO DA ANÁLISE E GERAÇÃO DE INSIGHT ---

--- INSIGHT ESTRATÉGICO ENCONTRADO ---
A **principal oportunidade de otimização (Dor Logística)** está na categoria 'relogios_presentes'. Esta categoria tem uma alta margem de 0.87, mas apresenta um tempo médio de entrega de **12.66 dias**, indicando que a lentidão logística está corroendo a lucratividade potencial.

Relatório visual salvo em: /content/drive/MyDrive/Projetos/Estagio_CBL_CienciaDeDados_HDL/reports/figures/margin_vs_delivery_report.png
